In [56]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.data.make_dataset import DICT_VARIABLES
from src.features import build_features

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax

In [71]:
cie_9 = pl.read_excel(
    "../data/external/Esquema_Registro-2023.xlsx",
    sheet_name="Anexo 2",
    read_csv_options={"skip_rows": 5},
)

cie_10 = pl.read_excel("../data/external/CIE-10 - sin_puntos_y_X.xlsx")

In [72]:
cie_10

Código,Descripción,Categoría,Sección,Capítulo
str,str,str,str,str
"""A00X""","""Cólera""","""A00 CÓLERA""","""A00-A09 ENFER…","""Cap.01 CIERTA…"
"""A000""","""Cólera debido …","""A00 CÓLERA""","""A00-A09 ENFER…","""Cap.01 CIERTA…"
"""A001""","""Cólera debido …","""A00 CÓLERA""","""A00-A09 ENFER…","""Cap.01 CIERTA…"
"""A009""","""Cólera, no esp…","""A00 CÓLERA""","""A00-A09 ENFER…","""Cap.01 CIERTA…"
"""A01X""","""Fiebres tifoid…","""A01 FIEBRES TI…","""A00-A09 ENFER…","""Cap.01 CIERTA…"
"""A010""","""Fiebre tifoide…","""A01 FIEBRES TI…","""A00-A09 ENFER…","""Cap.01 CIERTA…"
"""A011""","""Fiebre paratif…","""A01 FIEBRES TI…","""A00-A09 ENFER…","""Cap.01 CIERTA…"
"""A012""","""Fiebre paratif…","""A01 FIEBRES TI…","""A00-A09 ENFER…","""Cap.01 CIERTA…"
"""A013""","""Fiebre paratif…","""A01 FIEBRES TI…","""A00-A09 ENFER…","""Cap.01 CIERTA…"


## 1. Análisis Sociodemográfico de Pacientes Atendidos en el Hospital del Tórax

En este cuadernillo se quiere ver el perfil sociodemográfico de los pacientes hospitalizados
en el Hospital del Tórax. Se utilizará la base de datos pública de egresos hospitalarios DEIS
entre 2001 y 2020. Las preguntas que se quieren responder son:

- ¿Cómo es la distribución geográfica de los pacientes atendidos en el INT?
- ¿Cuál es el rango etario más atendido en el INT?
- ¿Cuál es el género más atendido en el INT?
- ¿Cuál es la previsión más atendida en el INT?
- ¿Cuál es la letra de FONASA más atendida en el INT?

Para responder estas preguntas se realizará una agrupación de los datos con todas las variables
sociodemográficas de interés para el análisis.

In [59]:
# Define todas las variables que se quieren saber

VARIABLES_SOCIODEMOGRAFICAS_ESTATICAS = ["ANO_EGRESO", "REGION_RESIDENCIA", "COMUNA_RESIDENCIA"]
VARIABLES_SOCIODEMOGRAFICAS_DINAMICAS = ["SEXO", "EDAD_CATEGORIA", "PREVISION", "BENEFICIARIO"]
VARIABLE_A_CONTAR = "DIAG1"

agrupacion_total = (
    VARIABLES_SOCIODEMOGRAFICAS_ESTATICAS
    + VARIABLES_SOCIODEMOGRAFICAS_DINAMICAS
    + [VARIABLE_A_CONTAR]
)

Luego, se genera la consulta de la base de datos. Para reducir el coste computacional, se filtra
la base de datos únicamente para los datos del INT. Posteriormente, se agrupa la base de datos
con las variables sociodemográficas de interés y se cuenta la ocurrencia de cada grupo.

In [73]:
desglose_sociodemografico = (
    pl.scan_csv("../data/processed/egresos_procesados.csv")
    .filter(pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR)
    .group_by(agrupacion_total)
    .count()
    .sort(["ANO_EGRESO"])
    .collect(streaming=True)
)

In [74]:
desglose_sociodemografico_cie = (
    desglose_sociodemografico.join(
        cie_9, how="left", left_on="COMUNA_RESIDENCIA", right_on="Código Comuna"
    )
    .with_columns(pl.col(["PREVISION", "Nombre Comuna"]).str.to_titlecase())
    .drop("Código Región")
)

desglose_sociodemografico_cie = desglose_sociodemografico_cie.join(
    cie_10, how="left", left_on="DIAG1", right_on="Código"
)

In [76]:
display(desglose_sociodemografico_cie.head())

ANO_EGRESO,REGION_RESIDENCIA,COMUNA_RESIDENCIA,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,Nombre Comuna,Nombre Región,Descripción,Categoría,Sección,Capítulo
i64,i64,i64,str,str,str,str,str,u32,str,str,str,str,str,str
2001,13,13201,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""J449""",1,"""Puente Alto""","""Metropolitana …","""Enfermedad pul…","""J44 OTRAS ENFE…","""J40-J47 ENFER…","""Cap.10 ENFERM…"
2001,13,13129,"""Mujer""","""(70, 80]""","""Fonasa""","""A""","""J459""",1,"""San Joaquín""","""Metropolitana …","""Asma, no espec…","""J45 ASMA""","""J40-J47 ENFER…","""Cap.10 ENFERM…"
2001,13,13114,"""Mujer""","""(80, 90]""","""Fonasa""","""B""","""J449""",2,"""Las Condes""","""Metropolitana …","""Enfermedad pul…","""J44 OTRAS ENFE…","""J40-J47 ENFER…","""Cap.10 ENFERM…"
2001,13,13127,"""Mujer""","""(50, 60]""","""Fonasa""","""A""","""Z136""",1,"""Recoleta""","""Metropolitana …","""Examen de pesq…","""Z13 EXAMEN DE …","""Z00-Z13 PERSO…","""Cap.21 FACTOR…"
2001,13,13123,"""Hombre""","""(60, 70]""","""Fonasa""","""A""","""J189""",1,"""Providencia""","""Metropolitana …","""Neumonía, no e…","""J18 NEUMONÍA, …","""J09-J18 GRIPE…","""Cap.10 ENFERM…"


En los resultados se indica el perfil sociodemográfico para cada combinación de variable presente
en la base de datos. A modo de ejemplo, hubo 1 ocurrencia en el 2001 de una paciente de la Región
número 13 y Comuna 13605 que tenía entre 70 a 80 años que tenía PREVISION FONASA Letra B diagnosticada
con el diagnóstico K810.

Uno de los diagnósticos de interés para el Instituto es el de código "C341".

In [78]:
display(desglose_sociodemografico_cie.filter(pl.col("DIAG1") == "C341"))
print(desglose_sociodemografico_cie.filter(pl.col("DIAG1") == "C341").select(pl.col("count").sum()))

ANO_EGRESO,REGION_RESIDENCIA,COMUNA_RESIDENCIA,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,Nombre Comuna,Nombre Región,Descripción,Categoría,Sección,Capítulo
i64,i64,i64,str,str,str,str,str,u32,str,str,str,str,str,str
2001,3,3102,"""Hombre""","""(50, 60]""","""Fonasa""","""A""","""C341""",1,"""Caldera""","""de Atacama""","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,13,13101,"""Hombre""","""(50, 60]""","""Fonasa""","""A""","""C341""",1,"""Santiago""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,3,3101,"""Hombre""","""(60, 70]""","""Fonasa""","""C""","""C341""",1,"""Copiapo""","""de Atacama""","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,13,13128,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""C341""",1,"""Renca""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,8,8101,"""Hombre""","""(60, 70]""","""Desconocido""",null,"""C341""",1,"""Concepción""","""del Biobío""","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,13,13106,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""C341""",1,"""Estación Centr…","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,13,13113,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""C341""",1,"""La Reina""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2003,6,6101,"""Hombre""","""(50, 60]""","""Fonasa""","""A""","""C341""",1,"""Rancagua""","""del Libertador…","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2003,13,13110,"""Mujer""","""(50, 60]""","""Fonasa""","""B""","""C341""",1,"""La Florida""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"


shape: (1, 1)
┌───────┐
│ count │
│ ---   │
│ u32   │
╞═══════╡
│ 1709  │
└───────┘


Por lo tanto, se muestra cada desglose sociodemográfico de los pacientes atendidos. Además,
se indica que la suma de cada conteo de grupo suma 1709 entre 2001 y 2020. Esto está en concordancia
con los egresos totales (1709) para el diagnóstico mencionado en el periodo mencionado.